In [1]:
import os
import sys
sys.path.append("../utils/")
sys.path.append("../models/")
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# from tf_models import UnetLike
from nn_utils import iterate_minibatches
from mulptiprocessing_utils import par_iterate_minibatches
from data_utils import load_files, random_nonzero_crops, augment, _reshape_to
from data_utils import combine, divide
from metrics import hausdorff, dice
#from model_controller import Model_controller
#from pytorch_utils import to_numpy, to_var, loss_cross_entropy, stochastic_step

%matplotlib inline

In [2]:
PATH = '../data/Utr/Utrecht/'
PATH1 = '../data/Sing/Singapore/'
PATH2 = '../data/Amst/GE3T/'

In [3]:
masks, t1, flairs, brain_mask = load_files(PATH)
brains = np.concatenate([flairs, t1], axis=1).astype(np.float32)

# masks1, t11, flairs1, brain_mask1 = load_files(PATH1)
# brains1 = np.concatenate([flairs1, t11], axis=1).astype(np.float32)

# masks2, t12, flairs2, brain_mask2 = load_files(PATH2)
# brains2 = np.concatenate([flairs2, t12], axis=1).astype(np.float32)

  0%|          | 0/20 [00:00<?, ?it/s]

INFO: brain mask from hardcoded path  ../data/skull_stripping


100%|██████████| 20/20 [00:05<00:00,  3.37it/s]


INFO: data reshape to hardcoded shape new_shape=! (256, 256, 84)


In [4]:
import tensorflow as tf

In [5]:
import time
import pickle
from sklearn.model_selection import KFold

skf = KFold(n_splits=4, random_state=0, shuffle=True)

In [6]:
import tensorflow as tf
import tensorflow.contrib.slim as slim


def batch_norm(t, momentum=0.9, axis=1, center=True, scale=True,
               training=False):
    assert axis == 1
    with tf.variable_scope('batch_norm'):
        shape = tf.shape(t)
        n_chans = t.get_shape().as_list()[1]

        t = tf.reshape(t, (shape[0], n_chans, 1, -1))
        t = slim.batch_norm(t, scale=scale, center=center, decay=momentum,
                            data_format='NCHW', fused=True,
                            is_training=training, scope='fused_batcn_norm_slim')
        t = tf.reshape(t, shape)
        return t


def activation(t):
    return tf.nn.relu(t)


class UnetLike:
    def __init__(self):
        self.x_ph = tf.placeholder(tf.float32, (None, 2, None, None, None))
        self.y_ph = tf.placeholder(tf.int32, (None, 1, None, None, None))
        self.training = tf.placeholder(tf.bool)
        self.lr = tf.placeholder(tf.float32)

        self.layers_down = [self.down_conv(16, 3), 
                            self.cnn_down_blocks(16, 3),
                            self.down_conv(32, 2, 2),
                            self.cnn_down_blocks(32, 64),
                            self.cnn_down_blocks(56, 64)]

        self.layers_inside = [self.cnn_down_blocks(56, 86), 
                              self.up_conv(64, 3)]

        self.layers_up = [self.up_conv(64, 3), self.up_conv(16, 3),
                          self.up_conv(32, 2, 2), self.up_conv(8, 3),
                          self.up_conv(3, 3)]

        self.pred = None
        self.summary = None
        self.loss = None
        self.train_op = None

    def build(self):
        with tf.variable_scope('model'):
            tensors = [self.x_ph]
            for i, apply_layer in enumerate(self.layers_down):
                name_ = 'down_' + str(i + 1)
                tensors.append(apply_layer(tensors[-1], name_, self.training))
            x = tensors[-1]
            for i, apply_layer in enumerate(self.layers_inside):
                name_ = 'inside_' + str(i)
                x = apply_layer(x, name_, self.training)
            for i, (apply_layer, left) in enumerate(
                    zip(self.layers_up, reversed(tensors))):
                name_ = 'up_' + str(i)
                x = tf.concat((left, x), 1)
                x = apply_layer(x, name_, self.training)
            self.pred = x

        self.loss = tf.losses.sparse_softmax_cross_entropy(
            tf.reshape(self.y_ph, [-1, 1]), tf.reshape(self.pred, [-1, 3]))

        with tf.name_scope('summary'):
            tf.summary.scalar('loss', self.loss)
            self.summary = tf.summary.merge_all()
        self.train_op = slim.learning.create_train_op(self.loss,
                                                      tf.train.AdamOptimizer(
                                                          learning_rate=self.lr))

    def validate_object(self, x, y, do_val_step):
        pass

    def predict_object(self, x, do_inf_step):
        pass

    # TODO clarify static or classmethod realization.
    def cnn_down_blocks(self, mid_channels, out_channels):
        """Like a bottleneck from resnet50, but without residual connections."""
        def res(t, name, training):
            with tf.variable_scope(name):                   
                channels = [mid_channels, mid_channels, out_channels]
                filters = [1, 3, 1]
                for i, (n_ch, n_filters) in enumerate(zip(channels, filters)):
                    with tf.variable_scope(name + 'down_conv_' + str(i)):
                        t = tf.layers.conv3d(t, n_ch, n_filters,
                                             data_format='channels_first',
                                             use_bias=False)
                        t = activation(batch_norm(t, training=training))
            return t
        return res

    def down_conv(self, n_channels, filter, strides=1):
        def conv(t, name, training):
            with tf.variable_scope(name):
                t = tf.layers.conv3d(t, n_channels, filter, strides=strides,
                                     data_format='channels_first',
                                     use_bias=False)
                t = batch_norm(t, training=training)
                t = activation(t)
            return t

        return conv

    # TODO clarify static or classmethod realization.
    def up_conv(self, n_channels, filter, strides=1):
        def conv(t, name, training):
            with tf.variable_scope(name):
                t = tf.layers.conv3d_transpose(t, n_channels, filter,
                                               strides=strides,
                                               data_format='channels_first',
                                               use_bias=False)
                # t = batch_norm(t, training=training)
                t = activation(t)
            return t

        return conv


In [ ]:
tf.reset_default_graph()

model = UnetLike()
model.build()
print ('ok')

graph = tf.get_default_graph()
file_writer = tf.summary.FileWriter('./log/new3', graph=graph, flush_secs=10)
    

ok


In [ ]:
counter = 0
k = 0
for tr_idx, ts_idx in skf.split(range(len(brains)), range(len(masks))):
    counter+=1
    X_train = brains[tr_idx]
    X_test = brains[ts_idx]
    Y_train = masks[tr_idx]
    Y_test = masks[ts_idx]

    with tf.Session(graph=graph) as session:
        session.run(tf.global_variables_initializer())
        epoches = 100
        print (epoches)
        for epoch in range(epoches):
            print (epoch)
            inputs, targets = random_nonzero_crops(X_train, Y_train, 400,
                                                   targ_shape=(50, 50, 40),
                                                   context_shape=(50, 50, 40))
            mean_loss, step = 0, 0
            bar = tqdm(iterate_minibatches(inputs, targets, 10, True),
                       total=inputs.shape[0] // 10)
            for x, y_true in bar:
                feed_dict = {model.x_ph: x.astype(np.float32), 
                             model.y_ph: y_true.astype(np.int32), 
                             model.training: True, 
                             model.lr: 0.01 / (epoch+1)**0.5}
                _, l ,p, loss, summary = session.run([model.train_op, model.lr, model.pred, 
                                                   model.loss, model.summary], 
                                                  feed_dict)
                print (l)
                file_writer.add_summary(summary, k)
                k += 1
                bar.set_description(str(loss)[:6])
                mean_loss += loss
    break

100
0


1.1157:   2%|▎         | 1/40 [00:01<01:05,  1.69s/it]

0.009999999776482582


1.0989:   5%|▌         | 2/40 [00:01<00:47,  1.25s/it]

0.009999999776482582


...

0.001280368771404028


1.0986:  98%|█████████▊| 39/40 [00:09<00:00,  4.16it/s]

0.001280368771404028


1.0986: 100%|██████████| 40/40 [00:09<00:00,  4.15it/s]

0.001280368771404028
61


In [2]:
!ls ./log/new3/events.out.tfevents.1499851336.neuro-t01

./log/new3/events.out.tfevents.1499851336.neuro-t01
